<a href="https://colab.research.google.com/github/niapollard/Benchmarking_QC/blob/main/052024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q jarvis-tools qiskit==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 25.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
!pip install qiskit_nature==0.6.2

In [ ]:
!pip install pyscf

In [ ]:
!pip install qiskit-aer==0.12.0

In [ ]:
import numpy as np

# Importing Qiskit and Jarvis libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.drivers.second_quantization import MethodType
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.transformers.second_quantization.electronic.active_space_transformer import ActiveSpaceTransformer
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
import itertools, functools
from qiskit.opflow import I, X, Y, Z
from jarvis.db.figshare import get_wann_electron, get_wann_phonon, get_hk_tb
from jarvis.core.circuits import QuantumCircuitLibrary

In [6]:
molecule = Molecule(
    geometry=[['Al',[0.00000000, 0.00000000, 0.00000000]]],
    charge=-1,
    multiplicity=1)

In [ ]:
def get_qubit_op(
    molecule,
    basis='sto3g',
    functional='lda',
    method= MethodType.RKS,
    driver_type= ElectronicStructureDriverType.PYSCF,
    converter= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]
  mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)
  res={}
  res['qubit_op']=qubit_op
  res['converter']=converter
  res['problem_reduced']=problem_reduced
  return res #qubit_op, converter, problem_reduced

def get_energy(optimizer='',device='',qubit_op='',seed=''):
  seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  print('device',device)
  print('seed',seed)

  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=1).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  # return eigenvalue, vqe, qi

  res={}
  res['eigenvalue']=eigenvalue
  res['vqe']=vqe
  res['qi']=qi
  return res

elements=['Al']
basis=['sto3g']
method=[MethodType.RKS]
#driver_type=[]
optimizer=[SLSQP(maxiter=1000)]
converter=[JordanWignerMapper()]
functions=['lda']
devices=[Aer.get_backend('statevector_simulator')]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          for n in converter:
            for o in functions:
              for p in devices:
                 print(i,j,k,m,n,o,p)

                 res1=get_qubit_op(molecule=molecule,basis=j,functional=o,method=k,converter=n)
                 converter=res1['converter']
                 res=get_energy(optimizer=m,device=p,qubit_op=res1['qubit_op'],seed=42)
                 print(res['eigenvalue'])
                 mem[i,j,k,m,n,o,p]=res['eigenvalue']
print(mem)

Al sto3g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7c200a0e83d0> <qiskit_nature.mappers.second_quantization.jordan_wigner_mapper.JordanWignerMapper object at 0x7c200a0ebb50> lda statevector_simulator


<ipython-input-13-c3f3b7a4fc1c>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-13-c3f3b7a4fc1c>:58: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-13-c3f3b7a4fc1c>:61: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
{('Al', 'sto3g', <MethodType.RKS: 'rks'>, <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7c200a0e83d0>, <qiskit_nature.mappers.second_quantization.jordan_wigner_mapper.JordanWignerMapper object at 0x7c200a0ebb50>, 'lda', StatevectorSimulator('statevector_simulator')): (8.5066017987e-08+0j)}
